In [115]:
import pyaudio
import wave

chunk = 1024 
sample_format = pyaudio.paInt16  
channels = 2
fs = 44100  
seconds = 5
filename = "output.wav"

p = pyaudio.PyAudio() 

print('Recording')

stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)

frames = []  
for i in range(0, int(fs / chunk * seconds)):
    data = stream.read(chunk)
    frames.append(data)

stream.stop_stream()
stream.close()
p.terminate()

print('Finished recording')

wf = wave.open(filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()

Recording
Finished recording


In [92]:


chunk = 1024  

wf = wave.open(filename, 'rb')

p = pyaudio.PyAudio()


stream = p.open(format = p.get_format_from_width(wf.getsampwidth()),
                channels = wf.getnchannels(),
                rate = wf.getframerate(),
                output = True)


data = wf.readframes(chunk)

while data != '':
    stream.write(data)
    data = wf.readframes(chunk)

stream.close()
p.terminate()

KeyboardInterrupt: 

In [82]:
import torch
import torchaudio
bundle = torchaudio.pipelines.HUBERT_ASR_LARGE


In [83]:
model = bundle.get_model()

In [84]:
labels = bundle.get_labels()

In [85]:
import os
import soundfile as sf


In [93]:
data, sample_rate = sf.read(filename, dtype='float32')
waveform = torch.from_numpy(data)
print(torchaudio.list_audio_backends())
if len(waveform.shape) > 1:
    waveform = waveform.mean(dim=1, keepdim=True)
waveform=waveform.squeeze(1)
waveform = waveform.unsqueeze(0)


[]


In [94]:
waveform=waveform.to('cpu')
emissions, _ = model(waveform)



In [95]:
#from torchaudio.models.decoder import CTCDecoderLM, CTCDecoderLMState
from pyctcdecode import build_ctcdecoder
log=emissions.detach().cpu()
logits = log.permute(0, 2, 1) 

In [96]:
# decoded_indices, _ = torchaudio.functional.ctc_decode(logits, blank=0, greedy=True)
from torchaudio.models.decoder import ctc_decoder
labels = ('-', 
 '|',
 'E',
 'T',
 'A',
 'O',
 'N',
 'I',
 'H',
 'S',
 'R',
 'D',
 'L',
 'U',
 'M',
 'W',
 'C',
 'F',
 'G',
 'Y',
 'P',
 'B',
 'V',
 'K',
 "'",
 'X',
 'J',
 'Q',
 'Z')

In [114]:
decoder = ctc_decoder(
    lexicon=None,        
    tokens=labels,
    beam_size=10,        
    lm=None,             
    sil_token='-',  
    blank_token='-'      
)
emissions = torch.randn(1, 100, len(labels))  
decoder.decode_begin()
decoder.decode_step(emissions[0, :50])  
decoder.decode_step(emissions[0, 50:])  
decoder.decode_end()
result = decoder.get_final_hypothesis()
print(f"Transcription: {result}")
hypothesis = result[0]
transcription = ''.join([labels[i] for i in hypothesis.tokens])
print("Final Transcription:", transcription)


Transcription: [CTCHypothesis(tokens=tensor([18, 20, 25, 23, 11, 12, 25, 15, 24, 16, 15,  4,  7,  4,  5, 25, 26, 16,
        18,  5, 20, 23, 15,  6, 23, 22,  1, 12, 13, 11, 12, 25, 28, 13, 23, 27,
         1, 12, 25, 26, 12,  1, 22,  1, 23, 18, 24,  9, 10, 18, 15, 26, 10, 25,
         9, 27,  1, 22, 26, 13, 24, 20, 15, 25, 13, 24, 12, 19, 16, 28, 23, 20,
        21, 12, 14, 25,  5,  6, 17,  7,  9,  1, 21, 16, 10,  5, 17,  7, 15, 12]), words=[], score=194.26401501893997, timesteps=tensor([ 1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20,
        21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
        39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 57, 58,
        59, 60, 61, 62, 63, 64, 65, 67, 69, 70, 71, 73, 74, 75, 76, 78, 79, 80,
        81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
       dtype=torch.int32))]
Final Transcription: GPXKDLXW'CWAIAOXJCGOPKWNKV|LUDLXZUKQ|LXJL|V|KG'SRGWJRXSQ|VJU'PWXU

In [1]:
import torch
import torchaudio
import soundfile as sf

bundle = torchaudio.pipelines.WAV2VEC2_ASR_BASE_960H
model = bundle.get_model()
labels = bundle.get_labels()

filename = 'output.wav'  
data, sample_rate = sf.read(filename, dtype='float32')
waveform = torch.from_numpy(data)

if waveform.ndim > 1:
    waveform = waveform.mean(dim=1)

if sample_rate != bundle.sample_rate:
    waveform = torchaudio.functional.resample(waveform, sample_rate, bundle.sample_rate)

waveform = waveform.unsqueeze(0)  # Shape: [1, time]

with torch.inference_mode():
    emissions, _ = model(waveform)

emission = torch.log_softmax(emissions[0], dim=-1)
tokens = torch.argmax(emission, dim=-1)

transcription = []
prev_token = None
blank_token = labels.index('|') 

for token in tokens:
    if token != prev_token and token != blank_token:
        transcription.append(labels[token])
    prev_token = token

transcribed_text = ''.join(transcription)
print("Transcription:", transcribed_text)


Transcription: -D-A-D--D-A-D--
